***Clasificación usando scikit-learn***

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>
Análisis diario de datos meteorológicos</p>

**En este cuaderno, utilizare scikit-learn para realizar una clasificación de datos meteorológicos basada en un árbol de decisiones.**

In [ ]:
!pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Ruta  archivo en Google Drive
archivo_path = '/content/drive/My Drive/diario.csv'

# Leo el archivo CSV
df = pd.read_csv(archivo_path)

# Primeras filas del DataFrame
df.head()


,number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
0,0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000,36.160000
1,1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0,24.328697,19.426597
2,2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000,14.460000
3,3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.0,0.0,12.189102,12.742547
4,4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.9,14730.0,92.410000,76.740000


In [5]:
df.columns

Index(['number', 'air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am', 'rain_duration_9am', 'relative_humidity_9am',
       'relative_humidity_3pm'],
      dtype='object')

In [6]:
df.head(10)

,number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
0,0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.00,0.0,42.420000,36.160000
1,1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.00,0.0,24.328697,19.426597
2,2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.00,20.0,8.900000,14.460000
3,3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.00,0.0,12.189102,12.742547
4,4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.90,14730.0,92.410000,76.740000
5,5,915.300000,78.404000,182.800000,9.932014,189.000000,10.983375,0.02,170.0,35.130000,33.930000
6,6,915.598868,70.043304,177.875407,3.745587,186.606696,4.589632,0.00,0.0,10.657422,21.385657
7,7,918.070000,51.710000,242.400000,2.527742,271.600000,3.646212,0.00,0.0,80.470000,74.920000
8,8,920.080000,80.582000,40.700000,4.518619,63.000000,5.883152,0.00,0.0,29.580000,24.030000
9,9,915.010000,47.498000,163.100000,4.943637,195.900000,6.576604,0.00,0.0,88.600000,68.050000


<br>Cada fila en daily_weather.csv captura datos del tiempo para un día separado.

Las mediciones de los sensores de la estación meteorológica se capturaron a intervalos de un minuto. Estas mediciones fueron procesadas para generar valores para describir el clima diario. Dado que este conjunto de datos se creó para clasificar los días de baja humedad frente a los días de no baja humedad (es decir, días con humedad normal o alta), las variables incluidas son las mediciones del clima en la mañana, y en la tarde. La idea es usar los valores del clima de la mañana para predecir si el día tendrá poca humedad o no en función de la medición de la humedad relativa de la tarde.

Qué son cada variable?

* **number:** Número único para cada fila
* **air_pressure_9am:** presión atmosférica entre las  8:55am y las 9:04am (*Hectopascales*)
* **air_temp_9am:** temperatura promedio del aire entre las 8:55am y las 9:04am (*Grados Fahrenheit*)
* **air_wind_direction_9am:** Dirección del  8:55am to 9:04am (*Grados a donde el 0 es el Norte e incrementa en sentido antihorario*)
* **air_wind_speed_9am:** Velocidad promedio del viento de 8:55am a 9:04am (*Millas por hora*)
* ** max_wind_direction_9am:** Dirección promedio de ráfagas de 8:55am a 9:10am
* **max_wind_speed_9am:** Velocidad promedio de las ráfagas de 8:55am a 9:04am (*Millas por hora*)
* **rain_accumulation_9am:** Cantidad de lluvia acumulada en las 24 horas previas a las 9am. (*Milímetros*)
* **rain_duration_9am:** Cantidad de tiempo en el que se registró la lluvia en las 24 horas previas a las 9 am (*segundos*)
* **relative_humidity_9am:** humedad relativa promedio de 8:55am a 9:04am (*por ciento*)
* **relative_humidity_3pm:** humedad relativa promedio de 2:55pm a 3:04pm (*por ciento*)
**texto en negrita**

In [7]:
df[df.isnull().any(axis=1)] #selecciono filas  (f i l a s por eso uso axis=1) que tengan valores null

,number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
16,16,917.890000,NaN,169.200000,2.192201,196.800000,2.930391,0.000,0.000000,48.990000,51.190000
111,111,915.290000,58.820000,182.600000,15.613841,189.000000,NaN,0.000,0.000000,21.500000,29.690000
177,177,915.900000,NaN,183.300000,4.719943,189.900000,5.346287,0.000,0.000000,29.260000,46.500000
262,262,923.596607,58.380598,47.737753,10.636273,67.145843,13.671423,0.000,NaN,17.990876,16.461685
277,277,920.480000,62.600000,194.400000,2.751436,NaN,3.869906,0.000,0.000000,52.580000,54.030000
334,334,916.230000,75.740000,149.100000,2.751436,187.500000,4.183078,NaN,1480.000000,31.880000,32.900000
358,358,917.440000,58.514000,55.100000,10.021491,NaN,12.705819,0.000,0.000000,13.880000,25.930000
361,361,920.444946,65.801845,49.823346,21.520177,61.886944,25.549112,NaN,40.364018,12.278715,7.618649
381,381,918.480000,66.542000,90.900000,3.467257,89.400000,4.406772,NaN,0.000000,20.640000,14.350000
409,409,NaN,67.853833,65.880616,4.328594,78.570923,5.216734,0.000,0.000000,18.487385,20.356594


limpieza de datos


No necesito un número para cada fila así que voy a  eliminar esa columna **texto en negrita**

In [8]:
del df['number'] #borro la columna number por que no la necesito

**Elimino los valores nulos**

In [9]:
before_rows =df.shape[0] #guardo la forma de la tabla antes de la limpieza (para comparar después)
print(before_rows)

1095


In [10]:
data = df.dropna()

In [11]:
after_rows = data.shape[0]
print(after_rows)

1064


**Aplico Clasificación**

In [12]:
clean_data = data.copy() #compio un dataframe en  uno nuevo
clean_data['high_humidity_label'] = (clean_data['relative_humidity_3pm'] > 24.99)*1
print(clean_data['high_humidity_label'])

0       1
1       0
2       0
3       0
4       1
       ..
1090    1
1091    1
1092    1
1093    1
1094    0
Name: high_humidity_label, Length: 1064, dtype: int64


**El objetivo lo guardamos en 'y'.**

In [13]:
y=clean_data[['high_humidity_label']].copy()
#y

In [14]:
y.head()

,high_humidity_label
0,1
1,0
2,0
3,0
4,1


**Usaremos las señales de los sensores de 9 am como características para predecir la humedad a las 3 pm**

In [15]:
morning_features = ['air_pressure_9am','air_temp_9am','avg_wind_direction_9am','avg_wind_speed_9am',
        'max_wind_direction_9am','max_wind_speed_9am','rain_accumulation_9am',
        ]

In [16]:
X = clean_data[morning_features].copy()

In [17]:
X.columns

Index(['air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am'],
      dtype='object')

In [18]:
y.columns

Index(['high_humidity_label'], dtype='object')

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

In [20]:
#type(X_train)
#type(X_test)
#type(y_train)
#type(y_test)
#X_train.head()
#y_train.describe()

**Ajusto**

In [39]:
#creo el clasificador
humidity_classifier = DecisionTreeClassifier(max_leaf_nodes=15, random_state=0)
#le paso los valores a entrenar
humidity_classifier.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=15, random_state=0)

In [22]:
type(humidity_classifier)

sklearn.tree._classes.DecisionTreeClassifier

Predict on Test Set

In [23]:
predictions = humidity_classifier.predict(X_test)
X_test

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am
456,918.800000,80.384000,189.600000,1.767183,80.300000,2.773806,0.000
845,921.613373,68.658973,70.703555,2.248932,96.844701,3.043049,0.000
693,917.900000,76.802000,154.100000,2.550112,199.400000,3.400149,0.000
259,914.830000,74.570000,175.500000,1.409272,153.800000,2.236940,0.000
723,917.010000,51.836000,130.200000,1.610597,159.900000,2.348787,0.000
...,...,...,...,...,...,...,...
46,917.900000,65.660000,183.700000,6.062107,188.500000,6.509495,0.021
116,919.561333,69.165521,137.144876,1.382894,123.019031,2.031607,0.000
799,917.650000,82.436000,108.000000,2.214571,62.800000,2.840914,0.000
350,917.120000,45.896000,179.200000,3.400149,195.000000,3.981753,0.000


In [24]:
predictions[:20]

array([0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0])

In [25]:
y_test['high_humidity_label'][:20]

,high_humidity_label
456,0
845,0
693,1
259,1
723,1
224,1
300,1
442,0
585,1
1057,1


**Mido la precisión**

In [26]:
accuracy_score(y_true = y_test, y_pred = predictions)

0.8210227272727273

**Usaremos las señales de los sensores de 9 am como características para predecir la humedad a las 3 pm**

In [27]:
morning_features = ['air_pressure_9am','air_temp_9am','avg_wind_direction_9am','avg_wind_speed_9am',
        'max_wind_direction_9am','max_wind_speed_9am','rain_accumulation_9am',
        ]

In [28]:
X = clean_data[morning_features].copy()

In [29]:
X.columns

Index(['air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am',
       'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am',
       'rain_accumulation_9am'],
      dtype='object')

In [30]:
y.columns

Index(['high_humidity_label'], dtype='object')

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

In [32]:
#type(X_train)
#type(X_test)
#type(y_train)
#type(y_test)
#X_train.head()
#y_train.describe()

**Ajusto**

In [46]:
#creo el clasificador,
humidity_classifier = DecisionTreeClassifier(max_leaf_nodes=15, random_state=0)
#le paso los valores a entrenar
humidity_classifier.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=15, random_state=0)

In [47]:
type(humidity_classifier)

sklearn.tree._classes.DecisionTreeClassifier

**Predict on Test Set**

In [48]:
predictions = humidity_classifier.predict(X_test)
X_test

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am
456,918.800000,80.384000,189.600000,1.767183,80.300000,2.773806,0.000
845,921.613373,68.658973,70.703555,2.248932,96.844701,3.043049,0.000
693,917.900000,76.802000,154.100000,2.550112,199.400000,3.400149,0.000
259,914.830000,74.570000,175.500000,1.409272,153.800000,2.236940,0.000
723,917.010000,51.836000,130.200000,1.610597,159.900000,2.348787,0.000
...,...,...,...,...,...,...,...
46,917.900000,65.660000,183.700000,6.062107,188.500000,6.509495,0.021
116,919.561333,69.165521,137.144876,1.382894,123.019031,2.031607,0.000
799,917.650000,82.436000,108.000000,2.214571,62.800000,2.840914,0.000
350,917.120000,45.896000,179.200000,3.400149,195.000000,3.981753,0.000


In [49]:
predictions[:20]

array([0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0])

In [50]:
predictions[:20]

array([0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0])

**Mido la precisión**

In [51]:
accuracy_score(y_true = y_test, y_pred = predictions)

0.8210227272727273